In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import sys
sys.path.append('..')

from src.solver import relocate_bicycles

path = '../data/processed/'

In [2]:
categories = pd.read_csv(f'{path}categories.csv')
a = categories.iloc[0].values
print('Disponibilidade: ', a)
s = categories.iloc[1].values
areas = [pd.read_csv(f'{path}area{i+1}.csv') for i in range(7)]
matrix = np.stack([area.T.values for area in areas], axis=0)
print('Shape Áreas: ', matrix.shape)

Disponibilidade:  [272. 270. 279. 267. 282. 279.]
Shape Áreas:  (7, 6, 115)


In [3]:
profit, opt_solution = relocate_bicycles(matrix, a, s, 10e12)
opt_solution

Status: optimal
Valor objetivo ótimo (Lucro esperado): 78866.16080000003

Solução ótima:
Número de bicicletas da categoria 2 a serem movidas para a área 1: 6.0
Número de bicicletas da categoria 1 a serem movidas para a área 3: 91.0
Número de bicicletas da categoria 2 a serem movidas para a área 3: 92.0
Número de bicicletas da categoria 3 a serem movidas para a área 3: 110.0
Número de bicicletas da categoria 4 a serem movidas para a área 3: 93.0
Número de bicicletas da categoria 5 a serem movidas para a área 3: 104.0
Número de bicicletas da categoria 6 a serem movidas para a área 3: 106.0
Número de bicicletas da categoria 1 a serem movidas para a área 4: 103.0
Número de bicicletas da categoria 2 a serem movidas para a área 4: 97.0
Número de bicicletas da categoria 3 a serem movidas para a área 4: 72.0
Número de bicicletas da categoria 4 a serem movidas para a área 4: 103.0
Número de bicicletas da categoria 5 a serem movidas para a área 4: 108.0
Número de bicicletas da categoria 6 a sere

array([[  0.,   6.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.],
       [ 91.,  92., 110.,  93., 104., 106.],
       [103.,  97.,  72., 103., 108., 109.],
       [  0.,   0.,   0.,   0.,   0.,   0.],
       [ 78.,  75.,  97.,  71.,  70.,  64.],
       [  0.,   0.,   0.,   0.,   0.,   0.]])

### 1 - Objective Function

In [4]:
def obj_function(solution, areas):
    profit = np.zeros_like(solution)
    for i, area in enumerate(areas):
        for j, column in enumerate(area.columns):
            profit[i][j] = sum(area[column][:int(solution[i][j])])
    return profit.sum()


profit = obj_function(opt_solution, areas)
profit

78866.1608

### 2 - Supplies Constraint

In [5]:
def sum_supplies(solution, supplies):
    sum = 0
    for categorie, supply in enumerate(supplies):
        sum+=int(solution[:,categorie].sum() > supply)
    return sum

sum_supplies(opt_solution, a)

0

### 3 - Disturbed Solutions

In [6]:
def generate_neighbor(solution):
    area1 = np.random.randint(0, solution.shape[0])
    area2 = np.random.randint(0, solution.shape[0])

    category = np.random.randint(0, solution.shape[1])

    max_bikes_to_move = min(solution[area1, category], solution[area2, category])
    bikes_to_move = np.random.randint(1, max_bikes_to_move + 2)

    neighbor_solution = np.copy(solution)
    neighbor_solution[area1, category] -= bikes_to_move
    neighbor_solution[area2, category] += bikes_to_move

    return np.clip(neighbor_solution, 0, None)

neighbor_solution = generate_neighbor(opt_solution)
neighbor_solution

array([[  0.,   6.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.],
       [ 91.,  92., 110.,  93., 104., 106.],
       [103.,  97.,  72., 103., 108., 109.],
       [  0.,   1.,   0.,   0.,   0.,   0.],
       [ 78.,  75.,  97.,  71.,  70.,  64.],
       [  0.,   0.,   0.,   0.,   0.,   0.]])

### 4 - Simulated Annealing

In [7]:
def simulated_annealing(initial_solution, initial_temperature, cooling_rate, num_iterations, areas, supplies):
    current_solution = initial_solution
    current_temperature = initial_temperature

    for i in range(num_iterations):
        # Avaliar a solução atual
        current_profit = obj_function(current_solution, areas)

        # Gerar uma solução vizinha
        neighbor_solution = generate_neighbor(current_solution)
        neighbor_profit = obj_function(neighbor_solution, areas)

        # Calcular a diferença de lucro entre as soluções
        profit_difference = neighbor_profit - current_profit

        # Aceitar a solução vizinha se ela for melhor ou de acordo com a probabilidade de Boltzmann
        if profit_difference > 0 or np.random.rand() < np.exp(profit_difference / current_temperature):
            if sum_supplies(neighbor_solution, supplies) == 0:
                current_solution = neighbor_solution

        # Reduzir a temperatura
        current_temperature *= cooling_rate

    return current_solution

In [8]:
initial_solution = np.random.randint(0, a.min()/len(a), size=(7, 6)).astype(float) 
initial_temperature = 1000
cooling_rate = 0.99
num_iterations = 5000

best_solution = simulated_annealing(initial_solution, initial_temperature, cooling_rate, num_iterations, areas, a)
print("Best Solution:\n", best_solution)
print("Profit: ", obj_function(best_solution, areas))

Best Solution:
 [[  0.   0.   0.   0.   1.   0.]
 [  0.   0.   0.   0.   0.   0.]
 [ 91.  92. 110.  89. 107. 106.]
 [103.  94.  77. 104. 108. 109.]
 [  0.   0.   0.   0.   0.   0.]
 [ 78.  84.  92.  74.  65.  64.]
 [  0.   0.   0.   0.   1.   0.]]
Profit:  78459.39600000001


### 5 - Initial Guess

In [9]:
for i in range(10):
    initial_solution = np.random.randint(0, a.min()/len(a), size=(7, 6)).astype(float)
    initial_temperature = 1000
    cooling_rate = 0.99
    num_iterations = 5000
    best_solution = simulated_annealing(initial_solution, initial_temperature, cooling_rate, num_iterations, areas, a)
    profit = obj_function(best_solution, areas)

    print('Profit: ', profit)

Profit:  78319.2982
Profit:  78424.1961
Profit:  78252.66010000001
Profit:  78425.0229
Profit:  78520.1029
Profit:  78650.44440000001
Profit:  78519.26699999999
Profit:  78576.2599
Profit:  78408.77729999999
Profit:  78161.64880000001


### 6 - Function

In [10]:
sys.path.append('..')

import src.algorithm as alg

In [11]:
np.random.seed(42)
initial_solution = np.random.randint(0, a.min()/len(a), size=(7, 6))
initial_temperature = 1000
cooling_rate = 0.99
num_iterations = 5000

profit, solution = alg.relocate_bicycles(initial_solution, initial_temperature, cooling_rate, num_iterations, areas, a)

Valor objetivo ótimo (Lucro esperado): 78636.76060000001

Solução ótima:
Número de bicicletas da categoria 1 a serem movidas para a área 3: 91.0
Número de bicicletas da categoria 2 a serem movidas para a área 3: 92.0
Número de bicicletas da categoria 3 a serem movidas para a área 3: 111.0
Número de bicicletas da categoria 4 a serem movidas para a área 3: 93.0
Número de bicicletas da categoria 5 a serem movidas para a área 3: 100.0
Número de bicicletas da categoria 6 a serem movidas para a área 3: 106.0
Número de bicicletas da categoria 1 a serem movidas para a área 4: 103.0
Número de bicicletas da categoria 2 a serem movidas para a área 4: 97.0
Número de bicicletas da categoria 3 a serem movidas para a área 4: 75.0
Número de bicicletas da categoria 4 a serem movidas para a área 4: 103.0
Número de bicicletas da categoria 5 a serem movidas para a área 4: 101.0
Número de bicicletas da categoria 6 a serem movidas para a área 4: 109.0
Número de bicicletas da categoria 1 a serem movidas para

In [12]:
solution

array([[  0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.],
       [ 91.,  92., 111.,  93., 100., 106.],
       [103.,  97.,  75., 103., 101., 109.],
       [  0.,   0.,   0.,   0.,   0.,   0.],
       [ 78.,  81.,  93.,  71.,  81.,  64.],
       [  0.,   0.,   0.,   0.,   0.,   0.]])